In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error


In [13]:

data = pd.read_csv("./data/housing.csv")

data.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [14]:
data.isnull().sum()


longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [15]:
data["total_bedrooms"].fillna(data["total_bedrooms"].median(), inplace=True)


C:\Users\Chandan Panda\AppData\Local\Temp\ipykernel_11724\1961396678.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["total_bedrooms"].fillna(data["total_bedrooms"].median(), inplace=True)


In [16]:
X = data.drop("median_house_value", axis=1)
y = data["median_house_value"]


In [17]:
num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object"]).columns

num_features, cat_features


(Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
        'total_bedrooms', 'population', 'households', 'median_income'],
       dtype='object'),
 Index(['ocean_proximity'], dtype='object'))

In [18]:
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ]
)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [20]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Decision Tree": DecisionTreeRegressor(max_depth=10, random_state=42)
}


In [21]:
results = []

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", model)
    ])
    
    pipeline.fit(X_train, y_train)
    
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    results.append([name, rmse_train, rmse_test, mae_test])


In [22]:
results_df = pd.DataFrame(
    results,
    columns=["Model", "RMSE (Train)", "RMSE (Test)", "MAE (Test)"]
)

results_df


,Model,RMSE (Train),RMSE (Test),MAE (Test)
0,Linear Regression,68433.937367,70060.521845,50670.738241
1,Ridge Regression,68434.995896,70067.346500,50677.170993
2,Decision Tree,48589.563780,61279.694223,40556.375765


##
Bias–Variance Analysis:

1. Linear Regression shows underfitting due to high bias. 
   Both training and testing RMSE are high, indicating inability to model non-linear relationships.

2. Decision Tree Regressor shows overfitting (high variance).
   Training RMSE is very low, but test RMSE increases significantly.

3. Ridge Regression provides the best bias–variance trade-off.
   Regularization controls variance while maintaining reasonable bias.

Conclusion:
Moderate model complexity with regularization generalizes best on limited data.
##
